In [13]:
import json, os, glob
from jsonschema import validate,exceptions
'''
NOTES: 
    keys must not have '_' in the first section. Use - instead. 
    Linked entries must not be nested with an "@nest"
'''



'''
cmip6plus/experiment
'''

def validate_json(data, schema, name = 'none', success=True):
    try:
        validate(instance=data, schema=schema)
        if success:
            print(f"Validation succeeded: {name}")
    except exceptions.ValidationError as err:
        print("Validation error:",name, err.message,)
        
miptag = os.popen('git describe --tags --abbrev=0').read().strip()
miptag

'v6.5.0.2'

In [14]:
def clean():
    # Walk through all directories and subdirectories
    for dirpath, _, filenames in os.walk('./'):
        for filename in filenames:
            if filename.endswith('.json') and filename != 'frame.json':
                file_path = os.path.join(dirpath, filename)
                try:
                    os.remove(file_path)
                    print(f"Removed: {file_path}")
                except Exception as e:
                    print(f"Failed to remove {file_path}: {e}")
                    
clean()

In [15]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@id": {"type": "string"},
        "@type": {"type": "string"},
        "table:table_id": {"type": "string"},
        "table:Conventions": {"type": "string"},
        "table:approx_interval": {"type": "number"},
        # "table:checksum": {"type": "string"},
        "table:cmor_version": {"type": "string"},
        "table:release": {"type": "string"},
        "table:data_specs_version": {"type": "string"},
        "table:generic_levels": {"type": "array"},
        "table:int_missing_value": {"type": "string"},
        "table:missing_value": {"type": "string"},
        "table:product": {
            "type": "object",
            "properties": {
                "@id": {"type": "string"}
            },
            "required": ["@id"]
        },
        "table:table_date": {"type": "string"},
        "table:variable_entry": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "@id": {"type": "string"}
                },
                "required": ["@id"]
            }
        
          
        }
    },
    "required": [
        "@id", "@type", "table:table_id", "table:Conventions", "table:approx_interval",
        "table:checksum", "table:cmor_version", "table:release", "table:data_specs_version",
        "table:generic_levels", "table:int_missing_value", "table:missing_value", 
        "table:product", "table:table_date", "table:variable_entry"
    ]
}

loc = f'./tables/schema.json'
json.dump(schema, open(loc,'w'),indent=4)

In [16]:
tables = glob.glob('../../Tables/*')
tables

for t in tables: 
    td = json.load(open(t))
    header = td['Header']
    lname = header['table_id'].lower()
    

    
    ld = {
        "@id":f"mip-cmor-tables:miptables/tables/{lname}",
        "@type":'cmip:dr-table',
        "table:table_id": header['table_id'],
        "table:Conventions": header['Conventions'],
        "table:approx_interval": header['approx_interval'],
        "table:checksum": header['checksum'],
        "table:cmor_version": header['cmor_version'],
        "table:release":miptag,
        "table:data_specs_version": header['data_specs_version'],
        "table:generic_levels": [i for i in header['generic_levels']],
        "table:int_missing_value": header['int_missing_value'],
        "table:missing_value": header['missing_value'],
        "table:product": {"@id": f"mip-cmor-tables:auxillary/product/{header['product'].lower()}"},
        "table:table_date": header['table_date'],
        "table:variable_entry":
            [{"@id":f"mip-cmor-tables:miptables/variables/table_specific/{lname}-{i.lower()}"} for i in td['variable_entry']]
    
    }
    

    validate_json(ld,schema,lname)

    loc = f'./tables/{lname}.json'
    json.dump(ld, open(loc,'w'),indent=4)

Validation succeeded: opfx
Validation succeeded: ap1hrpt
Validation succeeded: apdaylev
Validation succeeded: apsubhrpt
Validation succeeded: gigfx
Validation succeeded: opmonclimlev
Validation succeeded: lpday
Validation succeeded: simon
Validation succeeded: lp3hr
Validation succeeded: apmonlev
Validation succeeded: acmon
Validation succeeded: obyr
Validation succeeded: giayr
Validation succeeded: li6hrpt
Validation succeeded: lpyr
Validation succeeded: limon
Validation succeeded: obday
Validation succeeded: apdayz
Validation succeeded: op3hrpt
Validation succeeded: obmonlev
Validation succeeded: opmonz
Validation succeeded: ae1hr
Validation succeeded: giafx
Validation succeeded: ap1hr
Validation succeeded: lpfx
Validation succeeded: giamon
Validation succeeded: apday
Validation succeeded: ap3hr
Validation succeeded: ap6hrpt
Validation succeeded: aesubhrpt
Validation succeeded: aeday
Validation succeeded: ae6hrpt
Validation succeeded: apmonclimlev
Validation succeeded: gigyr
Validati

In [17]:
frame = {}

loc = f'./tables/frame.json'
json.dump(frame, open(loc,'w'),indent=4)

In [18]:
schema2 = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@id": {"type": "string"},
        "@type": {"type": "string"},
        "variable:long_name": {"type": "string"},
        "variable:out_name": {"type": "string"},
        "variable:positive": {"type": "string"},
        "variable:standard_name": {"type": "string"},
        "variable:type": {"type": "string"},
        "variable:units": {
            "type": "array",
            "items": {"type": "string"}
        }
    },
    "required": [
        "@id", "@type", "variable:long_name", "variable:out_name",
        "variable:positive", "variable:standard_name", "variable:type", "variable:units"
    ]
}


loc = f'./variables/generic/schema.json'
json.dump(schema2, open(loc,'w'),indent=4)


frame = {}

loc = f'./variables/generic/frame.json'
json.dump(frame, open(loc,'w'),indent=4)

In [19]:
schema3 = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@id": {"type": "string"},
        "@type": {"type": "string"},
        "variable:branded_variable_name": {"type": "string"},
        "variable:comment": {"type": "string"},
        "variable:frequency": {"type": "string"},
        "variable:dimensions": {
            "type": "array",
            "items": {"type": "string"}
        },
        "variable:modelling_realms": {
            "type": "array",
            "properties": {
                "@id": {"type": "string"}
            },
            "required": ["@id"]
        },
        "variable:provenance": {
            "type": "object",
            "properties": {
                "@id": {"type": "string"}
            },
            "required": ["@id"]
        },
        "variable:validation": {
            "type": "object",
            "properties": {
                "ok_max_mean_abs": {"anyOf": [{"type": "number"}, {"type": "string", "enum": [""]}]},
                "ok_min_mean_abs": {"anyOf": [{"type": "number"}, {"type": "string", "enum": [""]}]},
                "valid_max": {"anyOf": [{"type": "number"}, {"type": "string", "enum": [""]}]},
                "valid_min": {"anyOf": [{"type": "number"}, {"type": "string", "enum": [""]}]},
            },
            "required": ["ok_max_mean_abs", "ok_min_mean_abs", "valid_max", "valid_min"]
        }
    },
    "required": [
        "@id",
        "@type",
        "variable:branded_variable_name",
        "variable:comment",
        "variable:frequency",
        "variable:dimensions",
        "variable:modelling_realms",
        "variable:provenance",
        "variable:validation"
    ]
}
loc = f'./variables/table-specific/schema.json'
json.dump(schema3, open(loc,'w'),indent=4)


frame = {}

loc = f'./variables/table-specific/frame.json'
json.dump(frame, open(loc,'w'),indent=4)

In [20]:
loc = f'./provenance/schema.json'
json.dump({}, open(loc,'w'),indent=4)


frame = {}

loc = f'./provenance/frame.json'
json.dump(frame, open(loc,'w'),indent=4)

In [21]:

variables = json.load(open('../../MIP_variables.json'))
variables

for key,value in variables['variables'].items():
    
    lname = key.lower()
    
    ld = {
        "@id": f"mip-cmor-tables:miptables/variables/generic/{lname}",
        "@type": 'cmip:variable-generic',
        "variable:long_name": value['long_name'],
        "variable:out_name": value['out_name'],
        "variable:positive": value['positive'],
        "variable:standard_name": value['standard_name'],
        "variable:type": value['type'],
        "variable:units": value['units'].split()
    }
    
    
    validate_json(ld,schema2,lname,False)

    loc = f'./variables/generic/{lname}.json'
    json.dump(ld, open(loc,'w'),indent=4)
    

    for k,v in value['tables'].items():
    
        ln = f'{k.lower()}-{lname}'
        
        ld_data = {
        "@id": f"mip-cmor-tables:miptables/variables/table_specific/{ln}",
        "@type": "cmip:table-variable",
        "variable:generic":{"@id": f"mip-cmor-tables:miptables/variables/generic/{lname}"},
        "variable:branded_variable_name": v['branded_variable_name'],
        "variable:comment": v['comment'],
        "variable:frequency": v['frequency'],
        "variable:dimensions": v['dimensions'],
        "variable:modelling_realms": [{"@id": f"mip-cmor-tables:modellingrealms/{i}"} for i in v['modeling_realm']],
        "variable:provenance": {
            "@id":f"mip-cmor-tables:miptables/provenance/{ln}"
        },
        "variable:validation": {
            "ok_max_mean_abs": v['validation']['ok_max_mean_abs'],
            "ok_min_mean_abs": v['validation']['ok_min_mean_abs'],
            "valid_max": v['validation']['valid_max'],
            "valid_min": v['validation']['valid_min']
            }
        }
        
        
        
        validate_json(ld_data,schema3,lname,False)

        loc2 = f'./variables/table-specific/{ln}.json'
        json.dump(ld_data, open(loc2,'w'),indent=4)
            
            
            
        ldpro = {
            "@id":f"mip-cmor-tables:miptables/provenance/{ln}",
            "@type":"cmip:provenance",
            **v['provenance']
        }
        
        
        # validate_json(ld,schema4,lname,False)

        loc4 = f'./provenance/{ln}.json'
        json.dump(ldpro, open(loc4,'w'),indent=4)
            
        